In [5]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time
import logging

# Configure logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# IB API Client
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextOrderId = None
        self.data = []
        self.order_status = {}
        self.lock = threading.Lock()

    def nextValidId(self, orderId):
        self.nextOrderId = orderId
        logging.info(f"Next valid order ID: {self.nextOrderId}")

    def historicalData(self, reqId, bar):
        self.data.append(bar)

    def historicalDataEnd(self, reqId, start, end):
        logging.info("Historical data received")
        self.disconnect()

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld):
        logging.info(f"Order Status: ID={orderId}, Status={status}, Filled={filled}, Remaining={remaining}, AvgFillPrice={avgFillPrice}")
        with self.lock:
            self.order_status[orderId] = (status, filled, remaining, avgFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        logging.info(f"Open Order: ID={orderId}, Symbol={contract.symbol}, Action={order.action}, Quantity={order.totalQuantity}")

    def openOrderEnd(self):
        logging.info("All open orders received")


# Performance Metrics
class PerformanceMetrics:
    def __init__(self, returns):
        self.returns = returns

    def sharpe_ratio(self, risk_free_rate=0.01):
        return (np.mean(self.returns) - risk_free_rate) / np.std(self.returns)

    def sortino_ratio(self, risk_free_rate=0.01):
        downside_returns = self.returns[self.returns < 0]
        return (np.mean(self.returns) - risk_free_rate) / np.std(downside_returns)

    def max_drawdown(self):
        cumulative_returns = (1 + self.returns).cumprod()
        peak = cumulative_returns.cummax()
        drawdown = (cumulative_returns - peak) / peak
        return drawdown.min()

    def var(self, confidence_level=0.95):
        return np.percentile(self.returns, (1 - confidence_level) * 100)

    def cvar(self, confidence_level=0.95):
        var_value = self.var(confidence_level)
        return self.returns[self.returns <= var_value].mean()


# Trading Strategies with Order Execution
class TradingStrategies:
    def __init__(self, data, app):
        self.data = data
        self.app = app

####
    import pandas as pd
import numpy as np

class TradingStrategies:
    def __init__(self, data, app):
        self.data = data
        self.app = app

    def moving_average_crossover(self, short_window=50, long_window=200):
        signals = pd.DataFrame(index=self.data.index)
        signals['price'] = self.data['close']
        signals['short_mavg'] = self.data['close'].rolling(window=short_window, min_periods=1).mean()
        signals['long_mavg'] = self.data['close'].rolling(window=long_window, min_periods=1).mean()

        # Create the 'signal' column with all zeros initially
        signals['signal'] = 0.0

        # Set signals where the short moving average is greater than the long moving average
        signals.loc[short_window:, 'signal'] = np.where(
            signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0
        )

        # Generate trading orders (1.0 means buy, -1.0 means sell)
        signals['positions'] = signals['signal'].diff()

        self.execute_orders(signals)

        return signals




    def mean_reversion(self, window=20, z_entry_threshold=2.0, z_exit_threshold=0.5):
        signals = pd.DataFrame(index=self.data.index)
        signals['price'] = self.data['close']
        signals['moving_avg'] = self.data['close'].rolling(window=window, min_periods=1).mean()
        signals['moving_std'] = self.data['close'].rolling(window=window, min_periods=1).std()
        signals['z_score'] = (signals['price'] - signals['moving_avg']) / signals['moving_std']
        signals['signal'] = 0.0
        signals['signal'][signals['z_score'] > z_entry_threshold] = -1.0
        signals['signal'][signals['z_score'] < -z_entry_threshold] = 1.0
        signals['positions'] = signals['signal'].diff()

        self.execute_orders(signals)

        return signals

    def execute_orders(self, signals):
        for index, signal in signals.iterrows():
            if signal['positions'] == 1.0:
                self.place_order('BUY')
            elif signal['positions'] == -1.0:
                self.place_order('SELL')

   
    def place_order(self, action):
        contract = Contract()
        contract.symbol = "AAPL"
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"

        order = Order()
        order.action = action
        order.orderType = "MKT"
        order.totalQuantity = 100

        with self.app.lock:
            order_id = self.app.nextOrderId
            self.app.placeOrder(order_id, contract, order)
            self.app.nextOrderId += 1
            logging.info(f"Placed {action} order: ID={order_id}, Quantity=100")

# Main Trading Script with GUI
class TradingApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Trading Strategies with Order Execution")
        self.geometry("500x400")

        self.label = ttk.Label(self, text="Interactive Brokers Trading Metrics & Strategies", font=("Arial", 14))
        self.label.pack(pady=10)

        self.fetch_data_button = ttk.Button(self, text="Fetch Data", command=self.fetch_data)
        self.fetch_data_button.pack(pady=5)

        self.moving_average_button = ttk.Button(self, text="Moving Average Crossover Strategy", command=self.run_moving_average_crossover)
        self.moving_average_button.pack(pady=5)

        self.mean_reversion_button = ttk.Button(self, text="Mean Reversion Strategy", command=self.run_mean_reversion)
        self.mean_reversion_button.pack(pady=5)

        self.metrics_button = ttk.Button(self, text="Show Metrics", command=self.show_metrics)
        self.metrics_button.pack(pady=5)

        self.exit_button = ttk.Button(self, text="Exit", command=self.quit)
        self.exit_button.pack(pady=5)

        self.app = None

    def fetch_data(self):
        self.app = IBApi()
        self.app.connect("127.0.0.1", 7497, clientId=1)

        api_thread = threading.Thread(target=self.app.run, daemon=True)
        api_thread.start()

        contract = Contract()
        contract.symbol = "AAPL"
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"

        self.app.reqHistoricalData(1, contract, "", "1 Y", "1 day", "MIDPOINT", 1, 1, False, [])

        time.sleep(5)  # Sleep to allow data to be fetched

        data = [(bar.date, bar.close) for bar in self.app.data]
        self.df = pd.DataFrame(data, columns=['date', 'close']).set_index('date')

        self.returns = self.df['close'].pct_change().dropna()

        self.app.disconnect()

    def run_moving_average_crossover(self):
        if hasattr(self, 'df'):
            strategy = TradingStrategies(self.df, self.app)
            signals = strategy.moving_average_crossover()

            plt.figure(figsize=(12, 6))
            plt.plot(self.df.index, self.df['close'], label='Price')
            plt.plot(signals['short_mavg'], label='Short Moving Average (50 days)')
            plt.plot(signals['long_mavg'], label='Long Moving Average (200 days)')
            plt.plot(signals.loc[signals['positions'] == 1.0].index, signals['short_mavg'][signals['positions'] == 1.0], '^', markersize=10, color='g', lw=0, label='Buy Signal')
            plt.plot(signals.loc[signals['positions'] == -1.0].index, signals['short_mavg'][signals['positions'] == -1.0], 'v', markersize=10, color='r', lw=0, label='Sell Signal')
            plt.title('Moving Average Crossover Strategy')
            plt.legend()
            plt.show()
        else:
            messagebox.showerror("Error", "No data available. Please fetch data first.")

    def run_mean_reversion(self):
        if hasattr(self, 'df'):
            strategy = TradingStrategies(self.df, self.app)
            signals = strategy.mean_reversion()

            plt.figure(figsize=(12, 6))
            plt.plot(self.df.index, self.df['close'], label='Price')
            plt.plot(signals.loc[signals['positions'] == 1.0].index, self.df['close'][signals['positions'] == 1.0], '^', markersize=10, color='g', lw=0, label='Buy Signal')
            plt.plot(signals.loc[signals['positions'] == -1.0].index, self.df['close'][signals['positions'] == -1.0], 'v', markersize=10, color='r', lw=0, label='Sell Signal')
            plt.title('Mean Reversion Strategy')
            plt.legend()
            plt.show()
        else:
            messagebox.showerror("Error", "No data available. Please fetch data first.")

    def show_metrics(self):
        if hasattr(self, 'returns'):
            metrics = PerformanceMetrics(self.returns)

            sharpe_ratio = metrics.sharpe_ratio()
            sortino_ratio = metrics.sortino_ratio()
            max_drawdown = metrics.max_drawdown()
            var = metrics.var()
            cvar = metrics.cvar()

            metrics_text = f"""
            Sharpe Ratio: {sharpe_ratio:.2f}
            Sortino Ratio: {sortino_ratio:.2f}
            Max Drawdown: {max_drawdown:.2%}
            VaR (95%): {var:.2%}
            CVaR (95%): {cvar:.2%}
            """

            messagebox.showinfo("Performance Metrics", metrics_text)
        else:
            messagebox.showerror("Error", "No data available. Please fetch data first.")


#if __name__ == "__main__":
app = TradingApp()
app.mainloop()



C:\Users\Yodyium\AppData\Local\Temp\ipykernel_27088\2204404654.py:87: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 

AttributeError: '_tkinter.tkapp' object has no attribute 'df'